In [1]:
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn.metrics import roc_curve, auc
from scipy.spatial import distance



In [2]:
mock_seqtab=pd.read_csv("CC11.map.SeqTable.csv",index_col=0)
df_sv_list_names=['mock','hs_rdp_10398','hs_rdp_5224','hs_rdp_1017','hs_rdp_92','hs_rdp_12']
taxaFiles=["AbundanceOfTaxIdInSamples_primaryTaxid.csv"]+[d+ "_"+"oneRankEachSV_keepBest.csv" for d in df_sv_list_names[1:]]

In [3]:
# mock_seqtab=pd.read_csv("CC11.map.SeqTable.csv",index_col=0)

In [4]:
# df_sv_list_names=['mock','rdp_10398','rdp_5224','rdp_1017','rdp_92','rdp_12']


In [5]:
mock= pd.read_csv("AbundanceOfTaxIdInSamples_primaryTaxid.csv", index_col=0)
rdpa=pd.read_csv("hs_rdp_10398_oneRankEachSV_keepBest.csv",index_col=0)
rdpb=pd.read_csv("hs_rdp_5224_oneRankEachSV_keepBest.csv", index_col=0)
rdpc=pd.read_csv("hs_rdp_1017_oneRankEachSV_keepBest.csv", index_col=0)
rdpd=pd.read_csv("hs_rdp_92_oneRankEachSV_keepBest.csv", index_col=0)
rdpe=pd.read_csv("hs_rdp_12_oneRankEachSV_keepBest.csv", index_col=0)

In [6]:
mock.CC11CM15.describe()

count      65.000000
mean      752.615385
std      1057.560706
min         2.000000
25%        45.000000
50%       462.000000
75%       979.000000
max      5686.000000
Name: CC11CM15, dtype: float64

In [7]:
rdpb.CC11CM15.describe()

count      65.000000
mean      752.400000
std       951.587174
min         2.000000
25%        51.000000
50%       488.000000
75%      1013.000000
max      4878.000000
Name: CC11CM15, dtype: float64

In [8]:
rdpc.CC11CM15.describe()

count      65.000000
mean      752.400000
std       951.587174
min         2.000000
25%        51.000000
50%       488.000000
75%      1013.000000
max      4878.000000
Name: CC11CM15, dtype: float64

In [9]:
# taxaFiles=["AbundanceOfTaxIdInSamples_primaryTaxid.csv","rdp_10398_oneRankEachSV_keepBest.csv","rdp_5224_oneRankEachSV_keepBest.csv","rdp_1017_oneRankEachSV_keepBest.csv","rdp_92_oneRankEachSV_keepBest.csv","rdp_12_oneRankEachSV_keepBest.csv"]

In [10]:
taxaFiles

['AbundanceOfTaxIdInSamples_primaryTaxid.csv',
 'hs_rdp_10398_oneRankEachSV_keepBest.csv',
 'hs_rdp_5224_oneRankEachSV_keepBest.csv',
 'hs_rdp_1017_oneRankEachSV_keepBest.csv',
 'hs_rdp_92_oneRankEachSV_keepBest.csv',
 'hs_rdp_12_oneRankEachSV_keepBest.csv']

In [11]:
mock=mock_seqtab.copy()
mock.head()

,community,sourceSeq,seqID,organism,ncbi_tax_id,multiplicity
0,CC11CM0,AB036759.1.1480,CC11CM0SCR1e06de32f41c414aaa57f33949f4905c,Pseudoramibacter alactolyticus,113287,9
1,CC11CM0,AB253730.1.1456,CC11CM0SCR3e39a5fc61b6420cac1c2dd465292aec,Bacteroides barnesiae,376804,4656
2,CC11CM0,AB253731.1.1463,CC11CM0SCR3823cad73fba408b8b4a7cda1eb5e493,Bacteroides salanitronis,376805,1579
3,CC11CM0,AB298910.1.1471,CC11CM0SCR6f14135fa1ba41f49b480f6973684fb2,Eubacterium limosum,1736,1066
4,CC11CM0,AB510708.1.1476,CC11CM0SCRc168f76390fb4e7c9f4809f8d0100c39,Bacteroides stercoris,46506,684


In [12]:
for i, f in enumerate(taxaFiles):
    print(i,f)

0 AbundanceOfTaxIdInSamples_primaryTaxid.csv
1 hs_rdp_10398_oneRankEachSV_keepBest.csv
2 hs_rdp_5224_oneRankEachSV_keepBest.csv
3 hs_rdp_1017_oneRankEachSV_keepBest.csv
4 hs_rdp_92_oneRankEachSV_keepBest.csv
5 hs_rdp_12_oneRankEachSV_keepBest.csv


In [13]:
def prepareFiles():
    df_sv_list=[]
    for i, f in enumerate(taxaFiles):
        if i==0: #mock
            mock=mock_seqtab.copy()
            sample_ids=mock['community'].unique()
            sample_ids=['CC11CM'+str(i) for i in range(sample_ids.shape[0])]
            #not all tax_ids in the mock are primary
            translate={415850:1463164,195041:45634,592977:1680, 796939:796937, 41791:126333}
            mock.ncbi_tax_id.replace(translate, inplace=True)
            mock.rename(columns={'sourceSeq':'colind','organism':'tax_name','ncbi_tax_id':'tax_id'}, inplace=True)
            sv_ids = mock.colind.unique()
            temp=pd.DataFrame(index=sv_ids,columns=['tax_id']+sample_ids)
            for s in sample_ids:
                mock_s = mock[mock.community==s]
                mock_s.set_index('colind', inplace=True)
                temp.loc[mock_s.index, 'tax_id']=mock_s['tax_id']
                temp.loc[mock_s.index, s]=mock_s['multiplicity']
            temp=temp.fillna(0)
            
        else: #analyzed using dada2/pplacer/RDP
            temp=pd.DataFrame(index=sv_ids,columns=['tax_id']+sample_ids)
            temp1=pd.read_csv(f)
            #drop rank, taxa_name and colind (SV index)
            temp1 = temp1.loc[:,['colind','tax_id']+sample_ids]
            temp1.set_index('colind',inplace=True)
            temp.loc[temp1.index,'tax_id']=temp1['tax_id']
            temp.loc[temp1.index,sample_ids]=temp1[sample_ids]
            temp=temp.fillna(0)

        ##very strange tax_id for the mock is not duplicated but when I set the index as tax_id for the mock the index and the mock becomes duplicated!
        df_sv_list.append(temp)
        #print(temp.head())
    return df_sv_list

df_sv_list=prepareFiles()


In [14]:
dfmock = df_sv_list[0]

In [15]:
df_sv_list[0].to_csv("mock.csv"),df_sv_list[1].to_csv("rdp10398.csv"),df_sv_list[2].to_csv("rdp5224.csv"),df_sv_list[3].to_csv("rdp1017.csv"),df_sv_list[4].to_csv("rdp92.csv"),df_sv_list[5].to_csv("rdp12.csv"),

(None, None, None, None, None, None)

In [16]:
# df_sv_list[0].head(), df_sv_list[1].head(),df_sv_list[2], df_sv_list[3],df_sv_list[4], df_sv_list[5]

In [17]:
for i, f in enumerate(taxaFiles):
    print(i,f)

0 AbundanceOfTaxIdInSamples_primaryTaxid.csv
1 hs_rdp_10398_oneRankEachSV_keepBest.csv
2 hs_rdp_5224_oneRankEachSV_keepBest.csv
3 hs_rdp_1017_oneRankEachSV_keepBest.csv
4 hs_rdp_92_oneRankEachSV_keepBest.csv
5 hs_rdp_12_oneRankEachSV_keepBest.csv


In [18]:
def bray_curtis_distance(refindex, community):
    array1 = df_sv_list[0]["CC11CM"+community]
    array2 = df_sv_list[refindex]["CC11CM"+community]
    return(distance.braycurtis(array1,array2))
def bray_curtis_distance_df():
    d={1:"rdp10398", 2:"rdp5224",3:"rdp1017",4:"rdp92",5:"rdp12"}
    mock=mock_seqtab.copy()
    sample_ids=mock['community'].unique()
    sample_ids_list=['CC11CM'+str(i) for i in range(sample_ids.shape[0])]
    temp=pd.DataFrame(index=sample_ids_list,columns=["rdp10398", "rdp5224","rdp1017","rdp92","rdp12"])
    for i in range(1,6):
        for j in range(sample_ids.shape[0]):
            bcd=bray_curtis_distance(i, str(j))
            temp.loc["CC11CM"+str(j),d[i]]=bcd
    return (temp)
temp=bray_curtis_distance_df()
temp.to_csv("Bray-Curtis-Distance-HS.csv")           

In [19]:
temp


,rdp10398,rdp5224,rdp1017,rdp92,rdp12
CC11CM0,0,0,0,0.0325228,0
CC11CM1,0.0126526,0.0126526,0.0126526,0.0259614,0.0175728
CC11CM2,0.00219571,0.00219571,0.00219571,0.00219571,0.00219571
CC11CM3,0.00995201,0.00995201,0.00995201,0.00995201,0.00995201
CC11CM4,0.00617271,0.00617271,0.0115719,0.00617271,0.00617271
CC11CM5,0,0,0,0,0.0348701
CC11CM6,0,0,0,0,0
CC11CM7,0.00505239,0.00505239,0.00505239,0.00505239,0.00505239
CC11CM8,0,0,0,0.012328,0.0110071
CC11CM9,0,0,0,0,0


In [20]:
for i in range(100):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [21]:
bray_curtis_distance(1,"0")

0.0

In [22]:
refindex=1
community="0"

In [23]:
type(df_sv_list[0]["CC11CM"+community])

pandas.core.series.Series

In [24]:
    array1 = df_sv_list[0]["CC11CM"+community]
    array2 = df_sv_list[refindex]["CC11CM"+community]

In [25]:
array1["AB036759.1.1480"],array2["AB036759.1.1480"]

(9, 9.0)